In [3]:
import torch
import numpy as np
from PINN import PINNSolver

# Define the exact solution
def exact_solution(x, t):
    """Exact solution: u(x,t) = exp(-t) * sin(πx)"""
    return torch.exp(-t) * torch.sin(np.pi * x)

# Define the PDE equation (residual form)
def pde_equation(x_col, u_t, u_xx):
    """
    PDE residual: ∂u/∂t - ∂²u/∂x² - f(x,t) = 0
    where f(x,t) = exp(-t) * (sin(πx) - π²sin(πx))
    """
    x = x_col[:, 0:1]  # Extract x coordinate
    t = x_col[:, 1:2]  # Extract t coordinate
    
    # Source term
    f = torch.exp(-t) * (torch.sin(np.pi * x) - np.pi**2 * torch.sin(np.pi * x))
    
    # PDE residual
    residual = u_t - u_xx - f
    
    return residual

# Create PINN solver
pinn_solver = PINNSolver(
    steps=30000,
    learning_rate=1e-3,
    layers=np.array([2, 32, 32,32, 1]),  # 2 inputs (x,t), 2 hidden layers, 1 output (u)
    x_min=-1.0,
    x_max=1.0,
    t_min=0.0,
    t_max=1.0,
    total_points_x=200,
    total_points_t=100,
    num_boundary_points=100,
    num_collocation_points=10000,
    pde_equation=pde_equation,
    exact_solution=exact_solution
)

# Prepare training data
print("Preparing training data...")
pinn_solver.prepare_training_data()

# Train the model
pinn_solver.train()

# Visualize results
print("Generating visualizations...")
pinn_solver.visualize_results()


ImportError: cannot import name 'Image' from 'PIL' (c:\Users\ameli\Proyects\PINNs\.venv\Lib\site-packages\PIL\__init__.py)